首先我们创建一个数据库：

In [1]:
import sqlite3
# 创建数据库
sqllite_path = 'llmdb.db'
con = sqlite3.connect(sqllite_path)

# 创建表
sql = """
CREATE TABLE `section_stats` (
  `部门` varchar(100) DEFAULT NULL,
  `人数` int(11) DEFAULT NULL
);
"""
c = con.cursor()
cursor = c.execute(sql)
c.close()
con.close()

然后给数据库填充一些数据：

In [2]:
con = sqlite3.connect(sqllite_path)
c = con.cursor()
data = [
    ["专利部",22],
    ["商标部",25],
]
for item in data:
    sql = """
    INSERT INTO section_stats (部门,人数) 
    values('%s','%d')
    """%(item[0],item[1])
    c.execute(sql)
    con.commit()
c.close()
con.close()

> ⚠️ 为了简化操作，本次使用智谱 AI 的 glm-4-flash、glm-4-plus 和 embedding-3 ，请自行申请：https://www.bigmodel.cn/invite?icode=tzUaambCl5Rc0B964McaGGczbXFgPRGIalpycrEwJ28%3D
> 如使用本地大模型，请参考 tutorial/第06课-数据库对话Agent.md 。

In [3]:
# 我们先用requets库来测试一下大模型
import json
import requests
BASE_URL = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
# 请替换为您的智谱AI API_KEY
API_KEY = "YOUR_API_KEY"
# 构建请求头
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
    "Accept": "application/json"
}
payload = {
  "model": "glm-4-flash",
  "messages": [
    {
      "role": "user",
      "content": "请写一篇1000字左右的文章，论述法学专业的就业前景。"
    }
  ]
}
response = requests.post(BASE_URL, json=payload, headers=headers)
print(response.text)

{"choices":[{"finish_reason":"stop","index":0,"message":{"content":"法学专业的就业前景：挑战与机遇并存\n\n随着我国法治建设的不断推进，法学专业在高等教育中的地位日益凸显。法学专业毕业生在就业市场上具有广泛的需求，但同时面临着诸多挑战。本文将从法学专业的就业现状、就业前景以及应对策略三个方面进行论述。\n\n一、法学专业的就业现状\n\n1. 就业领域广泛\n\n法学专业毕业生可在政府机关、企事业单位、律师事务所、仲裁机构、法律援助机构等多个领域就业。其中，政府机关、企事业单位和律师事务所是法学专业毕业生的主要就业去向。\n\n2. 就业竞争激烈\n\n随着法学专业毕业生数量的逐年增加，就业竞争日益激烈。一方面，法学专业毕业生在求职过程中需要具备扎实的专业知识和实践经验；另一方面，用人单位对法学专业毕业生的综合素质要求越来越高。\n\n3. 就业地区差异明显\n\n法学专业毕业生在就业地区上存在明显差异。一线城市和发达地区的就业机会较多，薪资待遇较高；而二线及以下城市和欠发达地区的就业机会相对较少，薪资待遇较低。\n\n二、法学专业的就业前景\n\n1. 法治建设持续推进\n\n随着我国法治建设的不断推进，法律人才需求将持续增长。未来，法学专业毕业生在就业市场上将拥有更多的机会。\n\n2. 法律服务市场不断扩大\n\n随着市场经济的发展，法律服务市场不断扩大。企事业单位、个人对法律服务的需求日益增加，为法学专业毕业生提供了广阔的就业空间。\n\n3. 法律职业资格考试改革\n\n近年来，我国法律职业资格考试改革不断深化，考试难度和竞争激烈程度不断提高。通过法律职业资格考试的法学专业毕业生，将具备进入法律职业的资格，就业前景更加广阔。\n\n三、应对策略\n\n1. 提高自身综合素质\n\n法学专业毕业生应注重提高自身综合素质，包括专业知识、实践能力、沟通能力、团队协作能力等。通过参加实习、兼职、志愿服务等活动，积累实践经验，提升自身竞争力。\n\n2. 关注就业市场动态\n\n法学专业毕业生应关注就业市场动态，了解各行业、各地区的就业形势和需求。根据自身兴趣和特长，选择合适的就业方向。\n\n3. 培养跨学科知识\n\n法学专业毕业生应注重培养跨学科知识，如经济学、管理学、心理学等。这将有助于拓

如果想要流式输出，怎么办呢？

In [ ]:
# 我们先用requets库来测试一下大模型
import json
import requests
BASE_URL = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
# 请替换为您的智谱AI API_KEY
API_KEY = "YOUR_API_KEY"
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
    "Accept": "application/json"
}
payload = {
  "model": "glm-4-flash",
  "messages": [
    {
      "role": "user",
      "content": "请写一篇1000字左右的文章，论述法学专业的就业前景。"
    }
  ],
  "stream": True
}

response = requests.post(BASE_URL, json=payload, headers=headers, stream=True)  # 在这里设置stream=True告诉requests不要立即下载响应内容  
if response.status_code == 200:
    buffer = b''  # 用字节串缓存数据
    for chunk in response.iter_content():
        if chunk:
            buffer += chunk
            while b'\n\n' in buffer:  # 检查是否有完整的数据包
                # 分割数据包
                package, buffer = buffer.split(b'\n\n', 1)
                try:
                    # 尝试解码完整的数据包
                    package_str = package.decode('utf-8')
                    if package_str.startswith('data: '):
                        # 提取 JSON 部分
                        json_str = package_str.replace('data: ', '').strip()
                        data = json.loads(json_str)
                        # 提取 content
                        if data.get("choices") and data["choices"][0].get("delta", {}).get("content"):
                            content = data["choices"][0]["delta"]["content"]
                            print(content, end='', flush=True)  # 直接打印内容，不加前缀，实时刷新
                except (UnicodeDecodeError, json.JSONDecodeError):
                    continue
else:
    print(f"Error: {response.status_code}")

response.close()

法学专业的就业前景：挑战与机遇并存

随着我国法治建设的不断深化，法学专业的重要性日益凸显。法学专业作为一门综合性学科，不仅涉及法律理论，还涵盖法律实践，培养具备法律素养、专业能力和创新精神的高级法律人才。然而，在当前就业市场竞争激烈的环境下，法学专业的就业前景既有挑战，也充满机遇。

一、法学专业的就业挑战

1. 市场竞争激烈

近年来，法学专业毕业生人数逐年增加，导致就业市场竞争激烈。据统计，我国法学专业毕业生就业率一直处于较低水平。在求职过程中，法学专业毕业生需要面对众多竞争者，如其他专业毕业生、海外留学归国人员等。

2. 就业岗位有限

相较于其他热门专业，法学专业的就业岗位相对较少。一方面，我国法律职业资格考试（司法考试）通过率较低，导致从事法律职业的人数有限；另一方面，部分法学专业毕业生选择从事与法律无关的行业，进一步加剧了就业岗位的竞争。

3. 就业地域差异明显

法学专业毕业生在就业地域上存在明显差异。一线城市如北京、上海、广州等地，法学专业毕业生就业机会较多，薪资待遇相对较高；而二线、三线城市及农村地区，法学专业毕业生就业形势较为严峻。

二、法学专业的就业机遇

1. 法治建设需求旺盛

随着我国法治建设的不断推进，法律人才需求持续增长。政府、企事业单位、律师事务所等机构对法学专业毕业生的需求量较大，为法学专业毕业生提供了广阔的就业空间。

2. 法律服务行业蓬勃发展

法律服务行业是我国经济发展的重要支撑。近年来，法律服务行业呈现出蓬勃发展的态势，如律师事务所、仲裁机构、法律援助机构等。这些机构为法学专业毕业生提供了丰富的就业机会。

3. 法律职业教育市场潜力巨大

随着社会对法律人才素质要求的提高，法律职业教育市场潜力巨大。法学专业毕业生可以从事法律教育培训、咨询、研究等工作，满足社会对法律人才的需求。

4. 创业机会增多

近年来，我国政府鼓励创新创业，为法学专业毕业生提供了更多创业机会。例如，成立律师事务所、法律咨询公司、法律服务平台等，为法学专业毕业生提供了实现自我价值、实现职业发展的平台。

三、应对挑战，把握机遇

1. 提高自身综合素质

法学专业毕业生应注重提高自身综合素质，包括法律理论、实践能力、沟通能力、团队协作能力等。同时，关注行业动态，紧跟时代步伐，提高自己的竞争力。

2. 拓宽就业渠道

法学专业毕业生应拓

In [44]:
pip install llama-index-llms-zhipuai llama-index-embeddings-zhipuai -i https://mirrors.aliyun.com/pypi/simple

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [82]:
# 配置对话模型
from llama_index.llms.zhipuai import ZhipuAI

llm = ZhipuAI(model="glm-4-plus", api_key="YOUR API KEY")

In [80]:
# 配置Embedding模型
from llama_index.embeddings.zhipuai import ZhipuAIEmbedding

embedding = ZhipuAIEmbedding(model="embedding-3", api_key="YOUR API KEY")

In [83]:
# 测试对话模型
response = llm.complete("你是谁？")
print(response)

我是人工智能助手智谱清言（ChatGLM），是基于智谱 AI 公司于 2024 年训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。


In [81]:
# 测试嵌入模型
emb = embedding.get_text_embedding("你是谁？")
len(emb), type(emb)


(1024, list)

输出 (1024, list)

说明配置成功。

导入Llama-index相关的库，并配置对话模型和嵌入模型。

In [84]:
from llama_index.core.agent import ReActAgent  
from llama_index.core.tools import FunctionTool  
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings  
from llama_index.core.tools import QueryEngineTool   
from llama_index.core import SQLDatabase  
from llama_index.core.query_engine import NLSQLTableQueryEngine  
from sqlalchemy import create_engine, select  


# 配置默认大模型  
Settings.llm = llm
Settings.embed_model = embedding

这里的llm和embedding只要是llama-index支持的就行，有多种构建方法。详细可参见wow-rag课程的第二课。

In [86]:
# 创建数据库查询引擎  
engine = create_engine("sqlite:///llmdb.db")  
# prepare data  
sql_database = SQLDatabase(engine, include_tables=["section_stats"])  
query_engine = NLSQLTableQueryEngine(  
    sql_database=sql_database,   
    tables=["section_stats"],   
    llm=Settings.llm  
)

In [87]:
# 创建工具函数  
def multiply(a: float, b: float) -> float:  
    """将两个数字相乘并返回乘积。"""  
    return a * b  

multiply_tool = FunctionTool.from_defaults(fn=multiply)  

def add(a: float, b: float) -> float:  
    """将两个数字相加并返回它们的和。"""  
    return a + b

add_tool = FunctionTool.from_defaults(fn=add)

# 把数据库查询引擎封装到工具函数对象中  
staff_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="section_staff",
    description="查询部门的人数。"  
)

In [90]:
# 构建ReActAgent，可以加很多函数，在这里只加了加法函数和部门人数查询函数。
agent = ReActAgent.from_tools([add_tool, staff_tool], verbose=True)  
# 通过agent给出指令
response = agent.chat("请从数据库表中获取`专利部`和`商标部`的人数，并将这两个部门的人数相加！")  

> Running step e21793aa-2e53-43e6-82ba-1a567d26924a. Step input: 请从数据库表中获取`专利部`和`商标部`的人数，并将这两个部门的人数相加！
Thought: The current language of the user is: Chinese. I need to use a tool to help me answer the question.
Action: section_staff
Action Input: {'input': '专利部'}
Observation: 根据查询结果，专利部目前有22名员工。
> Running step 6ab94292-a8db-4ed5-816d-8d427ded8ba6. Step input: None
Thought: I have the number of employees in the 专利部. Now I need to get the number of employees in the 商标部.
Action: section_staff
Action Input: {'input': '商标部'}
Observation: 根据查询结果，商标部目前有25名员工。
> Running step 681c447c-1bee-4acd-98df-a0a832f4ca19. Step input: None
Thought: I have the number of employees in both the 专利部 and 商标部. Now I need to add these two numbers.
Action: add
Action Input: {'a': 22, 'b': 25}
Observation: 47
> Running step 728b4974-010d-437f-903f-0a58ef5f9880. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: 专利部和商标部的人数总和为47人。


理想输出:
```

Thought: 首先我需要使用section_staff工具来获取“专利部”和“商标部”的人数。
Action: section_staff
Action Input: {'input': '专利部'}
Observation: 根据查询结果，部门为“专利部”的统计数据共有22条。
Thought: 我还需要获取“商标部”的人数，我将再次使用section_staff工具。
Action: section_staff
Action Input: {'input': '商标部'}
Observation: 根据查询结果，部门为"商标部"的统计数据共有25条。
Thought: 我现在有了两个部门的人数：“专利部”有22人，“商标部”有25人。下一步我需要将这两个数字相加。
Action: add
Action Input: {'a': 22, 'b': 25}
Observation: 47
Thought: 我可以回答这个问题了，两个部门的人数之和是47人。
Answer: 专利部和商标部的总人数为47人。
```

In [89]:
print(response)

专利部和商标部的人数总和为47人。


理想输出:
专利部和商标部的人数总和为47人。

In [ ]:
注：目前这个功能不太稳定，上面这个结果看起来不错，但是是使用 glm-4-plus 才得到这个结果的。如果效果不好，需要换个更强的模型会更好。